In [6]:
import pandas as pd
import requests
import numpy as np

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

stateList = list(us_state_abbrev.keys())

URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'

r = requests.get(URL) 

with open("confirmed-jhu-timeseries.csv", "w") as fout:
    fout.write(r.content.decode('utf-8'))
    
URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'

r = requests.get(URL) 

with open("deaths-jhu-timeseries.csv", "w") as fout:
    fout.write(r.content.decode('utf-8'))
    
URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'

with open("recovered-jhu-timeseries.csv", "w") as fout:
    fout.write(r.content.decode('utf-8'))


In [2]:
df = pd.read_csv('confirmed-jhu-timeseries.csv')
df['Province/State'] = df['Province/State'].fillna('Total')
jhuConfirmed = df
df = df[df['Country/Region'].str.contains('US')]
confirmedDF = df[df['Province/State'].isin(stateList)]

df = pd.read_csv('deaths-jhu-timeseries.csv')
df['Province/State'] = df['Province/State'].fillna('Total')
jhuDeath = df
df = df[df['Country/Region'].str.contains('US')]
deathDF = df[df['Province/State'].isin(stateList)]

df = pd.read_csv('recovered-jhu-timeseries.csv')
df['Province/State'] = df['Province/State'].fillna('Total')
jhuRecovered = df
df = df[df['Country/Region'].str.contains('US')]
deathDF = df[df['Province/State'].isin(stateList)]

for state in stateList[0:1]:
    print(confirmedDF[confirmedDF['Province/State'].str.contains(state)].to_dict('r')[0])

{'Province/State': 'Alabama', 'Country/Region': 'US', 'Lat': 32.3182, 'Long': -86.9023, '1/22/20': 0, '1/23/20': 0, '1/24/20': 0, '1/25/20': 0, '1/26/20': 0, '1/27/20': 0, '1/28/20': 0, '1/29/20': 0, '1/30/20': 0, '1/31/20': 0, '2/1/20': 0, '2/2/20': 0, '2/3/20': 0, '2/4/20': 0, '2/5/20': 0, '2/6/20': 0, '2/7/20': 0, '2/8/20': 0, '2/9/20': 0, '2/10/20': 0, '2/11/20': 0, '2/12/20': 0, '2/13/20': 0, '2/14/20': 0, '2/15/20': 0, '2/16/20': 0, '2/17/20': 0, '2/18/20': 0, '2/19/20': 0, '2/20/20': 0, '2/21/20': 0, '2/22/20': 0, '2/23/20': 0, '2/24/20': 0, '2/25/20': 0, '2/26/20': 0, '2/27/20': 0, '2/28/20': 0, '2/29/20': 0, '3/1/20': 0, '3/2/20': 0, '3/3/20': 0, '3/4/20': 0, '3/5/20': 0, '3/6/20': 0, '3/7/20': 0, '3/8/20': 0, '3/9/20': 0, '3/10/20': 0, '3/11/20': 0, '3/12/20': 0, '3/13/20': 5, '3/14/20': 6, '3/15/20': 12, '3/16/20': 29, '3/17/20': 39, '3/18/20': 46, '3/19/20': 78, '3/20/20': 83, '3/21/20': 131, '3/22/20': 138, '3/23/20': 138.0}


In [3]:
import matplotlib
import matplotlib.pyplot as plt

doublingTime = 6.2
avgTTD = 17.3
caseMult = 2.0**(avgTTD/doublingTime)
mortalityRate = 0.034
infectedEst = caseMult / mortalityRate

timeSeries = jhuConfirmed.columns[4:]
estimated = jhuDeath.copy()
gap = jhuDeath.copy()
for colName in timeSeries:
    estimated[colName] = estimated[colName] * infectedEst
    
gap[timeSeries] = estimated[timeSeries].subtract(jhuConfirmed[timeSeries])
num = gap._get_numeric_data()
num[num < 0] = 0

country = 'Korea, South'
province = 'Total'

confirmedPlot = jhuConfirmed[jhuConfirmed['Province/State'].str.contains(province)]
deathPlot = jhuDeath[jhuDeath['Province/State'].str.contains(province)]
estimatedPlot = estimated[estimated['Province/State'].str.contains(province)]

matplotlib.style.use('ggplot')
ax = deathPlot[deathPlot['Country/Region'].str.contains(country)].round(0)[timeSeries].transpose().plot(kind='line', title ="Testing Gap", figsize=(15, 10), legend=True, fontsize=12)
confirmedPlot[confirmedPlot['Country/Region'].str.contains(country)].round(0)[timeSeries].transpose().plot(kind='line', title ="Testing Gap", figsize=(15, 10), legend=True, fontsize=12, ax=ax)
estimatedPlot[estimatedPlot['Country/Region'].str.contains(country)].round(0)[timeSeries].transpose().plot(kind='line', title ="Testing Gap", figsize=(15, 10), legend=True, fontsize=12, ax=ax)
ax.legend(['Deaths', 'Confirmed Infected', 'Estimated Infected'])
ax.set_xlabel("Day", fontsize=12)
ax.set_ylabel("People", fontsize=12)
plt.yscale("log")
plt.show()


<Figure size 1500x1000 with 1 Axes>

In [7]:
country = 'Korea, South'
province = 'Total'
tsValues = list(timeSeries)

confirmedDF = jhuConfirmed[jhuConfirmed['Country/Region'].str.contains(country) & jhuConfirmed['Province/State'].str.contains(province)]
confirmedTS = dict(confirmedDF[timeSeries].iloc[0])
deathDF = jhuDeath[jhuDeath['Country/Region'].str.contains(country) & jhuDeath['Province/State'].str.contains(province)]
deathTS = dict(deathDF[timeSeries].iloc[0])
recoveredDF = jhuRecovered[jhuRecovered['Country/Region'].str.contains(country) & jhuRecovered['Province/State'].str.contains(province)]
recoveredTS = dict(recoveredDF[timeSeries].iloc[0])
totalDeaths = list(deathTS.values())

newDeaths = list()
lastDeath = 0.0
for death in totalDeaths:
    newDead = death - lastDeath
    newDeaths.append(newDead)
    lastDeath = death
    
newDeathTS = dict(zip(tsValues, newDeaths))

pd.DataFrame([confirmedTS, recoveredTS, deathTS, newDeathTS], index=["Cumulative Confirmed", "Cumulative Recovered", "Cumulative Deaths", "New Deaths"])

TypeError: 'module' object is not callable